In [106]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np
import itertools
import warnings
from itertools import groupby
warnings.filterwarnings("ignore")

In [107]:
conn = psycopg2.connect(
host = "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c",
  password = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password",
  database = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db",
  port = '5432'
)

In [108]:
query = """select o.id as order_id,
            o.created_at::date as order_date,
            o.customer_id as user_id,
            o.total_line_items_price as total_price,
            O.total_discounts as total_discounts,
            (o.total_discounts * 100) / o.total_line_items_price as discount_percent,
            oli.product_id as product_id,
            oli.name as product_name,
            o.total_price as order_amount,
            CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            oli.quantity as quantity,
            p.product_type as product_category,
            oli.price as sp
            from orders o
            inner join order_line_item oli on oli.order_id = o.id
            inner join products p on oli.product_id=p.id"""

In [109]:
query1 = '''select
    created_at::date as OrderDate,
    id as order_id,
    customer_id as user_id,
    CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
        ELSE 'Delivered'
    END as order_status
    from orders;'''

query2 = '''select
    name as product_name,
    order_id,
    product_id,
    quantity,
    price,
    variant_title as variant
    from order_line_item;'''

query3 = '''select
    id as product_id,
    title as product_title,
    product_type as product_category,
    handle,
    tags
    from products;'''

In [110]:
data = pd.read_sql(query, conn)

In [111]:
data = data.drop_duplicates()
data = data.dropna()

In [112]:
e = pd.read_sql(query1, conn)
f = pd.read_sql(query2, conn)
g = pd.read_sql(query3, conn)

Metrics Calculations

In [113]:
data["order_date"] = pd.to_datetime(data["order_date"])
data[data["product_id"] == np.nan]
data = data.dropna()
data['WeekCount']=data['order_date'].dt.strftime('%Y-w%U')
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [114]:
data = data[data['order_status']=='Delivered']

In [115]:
WeekList = WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:5]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [116]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)

In [117]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New', 'Churned (Single Use Only)', 'Recent & Loyals', 'Champions', 'Churned (Repeat Use)']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero', 'High Value & Slow Moving', 'Low Value & Slow Moving', 'New Business Drivers', 'Repeat Business Drivers']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
lst = []

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()

    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
      if ((df['Frequency']==1) & (df['r_score']>2)):
          return "New"
      elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
          return 'Churned (Single Use Only)'
      elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
          return 'Recent & Loyals'
      elif ((df['Frequency'] > 1) & (df['m_score'] >=3) & (df['r_score'] > 2)):
          return 'Champions'
      else:
          return 'Churned (Repeat Use)'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
    ).reset_index()
    produser=data.groupby(['product_name','user_id']).agg(
        New_user=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    produser_newcount=produser.groupby('product_name').agg(
        New_count=pd.NamedAgg(column='New_user', aggfunc='sum')
    ).reset_index()
    data_repeat=data[data['TrxnRank']==2]
    produser_repeatcount=data_repeat.groupby('user_id').agg(
        Repeat_count=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    k=pd.merge(produser,produser_repeatcount,on='user_id',how='left')
    k.fillna(0)
    k1=k.groupby('product_name').agg(
        Repeat_count=pd.NamedAgg(column='Repeat_count', aggfunc='sum')
    ).reset_index()
    productlevel=pd.merge(product_level,produser_newcount, on='product_name', how='left')
    productlevel=pd.merge(productlevel,k1,on='product_name',how='left')

    productlevel['NewRepeatConversionRatio']=productlevel['Repeat_count']/productlevel['New_count']
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels)).astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero'
        elif df['NewRepeatConversion_score'] >2:
            return 'Repeat Business Drivers'
        elif (df['CP_score'] >2):
            return 'New Business Drivers'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving'
        else:
            return 'Low Value & Slow Moving'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x5 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    final_stats = final_stats.append(x5, ignore_index=True)
    print("done")
    break

  except:
    lst.append(x)


done


In [118]:
data = data[~(data['WeekCount'].isin(lst))]
output_list = [item for item in output_list if item not in lst]

In [119]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New', 'Churned (Single Use Only)', 'Recent & Loyals', 'Champions', 'Churned (Repeat Use)']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero', 'High Value & Slow Moving', 'Low Value & Slow Moving', 'New Business Drivers', 'Repeat Business Drivers']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
lst = []

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()

    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
      if ((df['Frequency']==1) & (df['r_score']>2)):
          return "New"
      elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
          return 'Churned (Single Use Only)'
      elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
          return 'Recent & Loyals'
      elif ((df['Frequency'] > 1) & (df['m_score'] >=3) & (df['r_score'] > 3) & (df['value_score'] > 2)):
          return 'Champions'
      else:
          return 'Churned (Repeat Use)'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
    ).reset_index()
    produser=data.groupby(['product_name','user_id']).agg(
        New_user=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    produser_newcount=produser.groupby('product_name').agg(
        New_count=pd.NamedAgg(column='New_user', aggfunc='sum')
    ).reset_index()
    data_repeat=data[data['TrxnRank']==2]
    produser_repeatcount=data_repeat.groupby('user_id').agg(
        Repeat_count=pd.NamedAgg(column='user_id', aggfunc='nunique')
    ).reset_index()
    k=pd.merge(produser,produser_repeatcount,on='user_id',how='left')
    k.fillna(0)
    k1=k.groupby('product_name').agg(
        Repeat_count=pd.NamedAgg(column='Repeat_count', aggfunc='sum')
    ).reset_index()
    productlevel=pd.merge(product_level,produser_newcount, on='product_name', how='left')
    productlevel=pd.merge(productlevel,k1,on='product_name',how='left')

    productlevel['NewRepeatConversionRatio']=productlevel['Repeat_count']/productlevel['New_count']
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels)).astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero'
        elif df['NewRepeatConversion_score'] >2:
            return 'Repeat Business Drivers'
        elif (df['CP_score'] >2):
            return 'New Business Drivers'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving'
        else:
            return 'Low Value & Slow Moving'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x5 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    final_stats = final_stats.append(x5, ignore_index=True)

  except:
    pass

In [120]:
def group_consecutive(lst):
    result = []
    current_group = []

    for key, group in groupby(lst):
        key = next(group)

        if current_group and current_group[-1] == key:
            continue

        current_group.append(key)

    result.extend(current_group)
    return result

product_profile_df = final_stats.groupby(['product_name', 'product_id']).agg(
    Total_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
    ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
    Total_Orders=pd.NamedAgg(column='order_id', aggfunc='nunique'),
    Journey=pd.NamedAgg(column='Product Cohort Lables', aggfunc=lambda x: group_consecutive(x.tolist())),
    First_Tag=pd.NamedAgg(column='Product Cohort Lables', aggfunc=lambda x: group_consecutive(x.tolist())[0]),
    Last_Tag=pd.NamedAgg(column='Product Cohort Lables', aggfunc=lambda x: group_consecutive(x.tolist())[-1]),
    Tag_Count=pd.NamedAgg(column='Product Cohort Lables', aggfunc=lambda x: x.nunique()),
    Asp_Count=pd.NamedAgg(column='sp', aggfunc=lambda x: x.nunique()),
    Asp_Lst=pd.NamedAgg(column='sp', aggfunc=lambda x: list(set(x.astype(str)))),
).reset_index()

product_profile_df["Cart_Penetration_Overall"] = product_profile_df["Total_Orders"] / final_stats["order_id"].nunique()


In [121]:
product_profile_df.to_csv("product_profile_dfdf.csv", index=False)

1-2, 2-3, 3-4, Conversions

In [122]:
# product_name, product_id, variant, product_title, product_category, handle, tags
parameter_input = "product_name"

In [123]:
e = e[e['order_status']=='Delivered']
e.drop('order_status',axis=1,inplace=True)

In [124]:
final_stats.to_csv("final_stats_toffee.csv")

In [125]:
e_sorted = e.sort_values(by=['user_id', 'orderdate', 'order_id'])
# e['Transaction_Rank'] = e.groupby(['user_id'])['orderdate'].rank(method='first')

In [126]:
merged_df = pd.merge(f, e, on='order_id')
merged_df.dropna(inplace = True)
merged_df['Transaction_Rank'] = merged_df.groupby(['user_id',parameter_input])['orderdate'].rank(method='first')

In [127]:
final = pd.merge(merged_df, g, on='product_id')

In [128]:
final['Total Order Value'] = final['quantity'] * final['price']

In [129]:
def main(final, transactionx, transactiony, parameter):

    final1 = final.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    if parameter == "tags":
        final_tags = final1.copy()
        final_tags['tags'] = final_tags['tags'].str.split(',')
        final_tags = final_tags.explode('tags', ignore_index=True)
        final1 = final_tags
    else:
        pass

    #-------------------------------------------------------------------------------------------------------------------------------

    trans_x = final1[final1['Transaction_Rank']==transactionx]
    trans_y = final1[final1['Transaction_Rank']==transactiony]

    #-------------------------------------------------------------------------------------------------------------------------------

    xuser_count = trans_x.groupby(parameter).agg(
        x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()

    yuser_count = trans_y.groupby(parameter).agg(
        y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()


    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on=parameter, how='left')

    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result = main_grouped_for_title.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Conversion_Rate_x_to_y'] = result['y_trxn_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Cart_Penetration_for_x_transaction'] = result['x_trxn_pool']/(result['x_trxn_pool'].sum())
    result['Cart_Penetration_for_y_transaction'] = result['y_trxn_pool']/(result['y_trxn_pool'].sum())

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result.copy()
    result2 = result.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result1.sort_values(by='Conversion_Rate_x_to_y', ascending=False)
    result2 = result2.sort_values(by='Cart_Penetration_for_x_transaction', ascending=False)

    #-------------------------------------------------------------------------------------------------------------------------------

    result1['Mix']= result1['Conversion_Rate_x_to_y']*100/result1['Conversion_Rate_x_to_y'].sum()
    result1['CumuSum']= result1['Mix'].cumsum()
    result1['Conversion Rate Bin']=""
    result1.loc[result1['CumuSum']<=75,"Conversion Rate Bin"]='HighConversion'
    result1.loc[(result1['CumuSum']>75) & (result1['CumuSum']<=101),"Conversion Rate Bin"]='LowConversion'

    result2['Mix']= result2['Cart_Penetration_for_x_transaction']*100/result2['Cart_Penetration_for_x_transaction'].sum()
    result2['CumuSum']= result2['Mix'].cumsum()
    result2['Cart Penetration Bin']=""
    result2.loc[result2['CumuSum']<=75,"Cart Penetration Bin"]='HighCartPenetration'
    result2.loc[(result2['CumuSum']>75) & (result2['CumuSum']<=101),"Cart Penetration Bin"]='LowCartPenetration'

    #-------------------------------------------------------------------------------------------------------------------------------

    result2 = result2[[parameter,'Cart Penetration Bin']]
    final_merged_on_title_x_to_y = pd.merge(result1, result2, on=parameter)

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y = final_merged_on_title_x_to_y[final_merged_on_title_x_to_y['Conversion Rate Bin'] != 0]

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y['Bin'] = final_merged_on_title_x_to_y['Conversion Rate Bin'] + '_' + final_merged_on_title_x_to_y['Cart Penetration Bin']
    # final_merged_on_title_x_to_y.drop(['Conversion Rate Bin','Cart Penetration Bin'], axis=1, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    merged_title_on_bin = final_merged_on_title_x_to_y.groupby('Bin').agg({
        parameter: 'nunique'
    }).reset_index()

    #-------------------------------------------------------------------------------------------------------------------------------

    x = final_merged_on_title_x_to_y.columns
    y = []
    for i in x:
        if "x_" in i:
            i = i.replace("x_", f"{str(transactionx)}_")
        if "_x" in i:
            i = i.replace("_x", f"_{str(transactiony)}")
        if "y_" in i:
            i = i.replace("y_", f"{str(transactiony)}_")
        if "_y" in i:
            i = i.replace("_y", f"_{str(transactiony)}")
        y.append(i)
    final_merged_on_title_x_to_y.columns = y

    #-------------------------------------------------------------------------------------------------------------------------------

    return merged_title_on_bin, final_merged_on_title_x_to_y[[parameter, f"Conversion_Rate_{transactionx}_to_{transactiony}", f"Cart_Penetration_for_{transactionx}_transaction", f"Cart_Penetration_for_{transactiony}_transaction"]]

In [130]:
bins, product_data = main(final, 1, 2, parameter_input)
for i in range (2, 4):
    bins, input = main(final, i, i+1, parameter_input)
    product_data = pd.merge(product_data, input, on = parameter_input, how = 'left', suffixes=('', '_remove'))
product_data.drop([i for i in product_data.columns if 'remove' in i], axis=1, inplace=True)

In [131]:
product_profile_final = pd.merge(product_profile_df, product_data, on="product_name", how="left")
product_profile_final.fillna(0, inplace=True)

In [132]:
product_profile_final

,product_name,product_id,Total_Value,ASP,Total_Orders,Journey,First_Tag,Last_Tag,Tag_Count,Asp_Count,Asp_Lst,Cart_Penetration_Overall,Conversion_Rate_1_to_2,Cart_Penetration_for_1_transaction,Cart_Penetration_for_2_transaction,Conversion_Rate_2_to_3,Cart_Penetration_for_3_transaction,Conversion_Rate_3_to_4,Cart_Penetration_for_4_transaction
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,5100.0,1700.000000,3,[Repeat Business Drivers],Repeat Business Drivers,Repeat Business Drivers,1,1,[1700.0],0.000048,0.000000,0.000041,0.000000,0.00,0.000000,0.000000,0.000000
1,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,42900.0,1950.000000,22,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,1,[1950.0],0.000352,0.047619,0.000287,0.000144,0.00,0.000000,0.000000,0.000000
2,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,12100.0,2016.666667,6,[High Value & Slow Moving],High Value & Slow Moving,High Value & Slow Moving,1,2,"[2350.0, 1950.0]",0.000096,0.000000,0.000082,0.000000,0.00,0.000000,0.000000,0.000000
3,Aeropress Coffee Maker,5321695592586,68700.0,3816.666667,18,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,2,"[4100.0, 3800.0]",0.000288,0.000000,0.000096,0.000000,0.00,0.000000,0.000000,0.000000
4,Aeropress Filter Paper,7736482594954,1040.0,520.000000,2,[Low Value & Slow Moving],Low Value & Slow Moving,Low Value & Slow Moving,1,1,[520.0],0.000032,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,259700.0,490.000000,510,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,[490.0],0.008157,0.051760,0.006609,0.003604,0.12,0.001784,0.666667,0.002899
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,150150.0,650.000000,195,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,[650.0],0.003119,0.085714,0.002394,0.002163,0.40,0.003567,0.166667,0.001449
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,23400.0,1300.000000,15,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,[1300.0],0.000240,0.000000,0.000205,0.000000,0.00,0.000000,0.000000,0.000000
911,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7542394617994,61850.0,1508.536585,41,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,3,"[1250.0, 1150.0, 1550.0]",0.000656,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [133]:
product_profile_final.to_csv("product_profile_new_new_new.csv", index=False )

In [134]:
data

,order_id,order_date,user_id,total_price,total_discounts,discount_percent,product_id,product_name,order_amount,order_status,...,product_category,sp,WeekCount,TrxnRank,New,Repeat,ordervalue,AOVContribution,DeltaDaysinOrders,DeltaOrderValues
27,2423748001930,2020-09-20,3.562720e+12,650.0,195.0,30.0,5321724231818,Easy Brew - Cold Brew Bags (Pack of 4),455.0,Delivered,...,Cold Brew,650.0,2020-w38,1.0,1,0,650.0,0.010391,NaN,NaN
28,2424323997834,2020-09-20,3.563596e+12,650.0,195.0,30.0,5321724231818,Easy Brew - Cold Brew Bags (Pack of 4),455.0,Delivered,...,Cold Brew,650.0,2020-w38,1.0,1,0,650.0,0.010391,NaN,NaN
29,2424320131210,2020-09-20,3.563591e+12,650.0,195.0,30.0,5321724231818,Easy Brew - Cold Brew Bags (Pack of 4),455.0,Delivered,...,Cold Brew,650.0,2020-w38,1.0,1,0,650.0,0.010391,NaN,NaN
30,2424245485706,2020-09-20,3.563472e+12,650.0,195.0,30.0,5321724231818,Easy Brew - Cold Brew Bags (Pack of 4),455.0,Delivered,...,Cold Brew,650.0,2020-w38,1.0,1,0,650.0,0.010391,NaN,NaN
31,2424237817994,2020-09-20,3.563460e+12,650.0,195.0,30.0,5321724231818,Easy Brew - Cold Brew Bags (Pack of 4),455.0,Delivered,...,Cold Brew,650.0,2020-w38,1.0,1,0,650.0,0.010391,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82878,5363883180170,2023-12-25,6.932787e+12,299.0,0.0,0.0,5321601712266,South Indian Traditional Filter Coffee - Dark ...,324.0,Delivered,...,South Indian Filter Coffee,299.0,2023-w52,2.0,0,1,299.0,0.004780,18.0,0.0
82879,5363879313546,2023-12-25,6.970226e+12,299.0,0.0,0.0,5321601712266,South Indian Traditional Filter Coffee - Dark ...,324.0,Delivered,...,South Indian Filter Coffee,299.0,2023-w52,1.0,1,0,299.0,0.004780,0.0,-100.0
82880,5363873513610,2023-12-25,6.965660e+12,299.0,0.0,0.0,5321601712266,South Indian Traditional Filter Coffee - Dark ...,299.0,Delivered,...,South Indian Filter Coffee,299.0,2023-w52,1.0,1,0,299.0,0.004780,NaN,NaN
82881,5363869188234,2023-12-25,6.970219e+12,398.0,0.0,0.0,5321601712266,South Indian Traditional Filter Coffee - Dark ...,423.0,Delivered,...,South Indian Filter Coffee,199.0,2023-w52,1.0,1,0,398.0,0.006363,NaN,NaN


In [135]:
data['Discount_Tag'] = data['total_discounts'].apply(lambda x: 'without_discount' if x == 0 else 'with_discount')

In [136]:
grouped_data = data.groupby(['product_id', 'product_name', 'Discount_Tag'])
average_price = grouped_data['total_price'].mean().reset_index()
result = average_price.pivot(index=['product_id', 'product_name'], columns='Discount_Tag', values='total_price').reset_index()
result.columns = ['product_id', 'product_name', 'Average_Price_With_Discount', 'Average_Price_Without_Discount']

In [137]:
def get_discount_list(discounts):
    return discounts.tolist()

grouping = data.groupby(['product_name', 'product_id']).agg(
    Discount_List=pd.NamedAgg(column='total_discounts', aggfunc=get_discount_list),
).reset_index()

In [138]:
grouping['Order_Count_Without_Discount'] = 0
grouping['Order_Count_With_Discount'] = 0
def count_positive_discounts(discount_list):
    return len([value for value in discount_list if isinstance(value, (int, float)) and value > 0.0])

def count_zero_discounts(discount_list):
    return len([value for value in discount_list if isinstance(value, (int, float)) and value == 0.0])
for index, row in grouping.iterrows():
    discount_list = row['Discount_List']
    grouping['Order_Count_With_Discount'] = grouping['Discount_List'].apply(count_positive_discounts)
    grouping['Order_Count_Without_Discount'] = grouping['Discount_List'].apply(count_zero_discounts)

In [139]:
product_profile_final

,product_name,product_id,Total_Value,ASP,Total_Orders,Journey,First_Tag,Last_Tag,Tag_Count,Asp_Count,Asp_Lst,Cart_Penetration_Overall,Conversion_Rate_1_to_2,Cart_Penetration_for_1_transaction,Cart_Penetration_for_2_transaction,Conversion_Rate_2_to_3,Cart_Penetration_for_3_transaction,Conversion_Rate_3_to_4,Cart_Penetration_for_4_transaction
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,5100.0,1700.000000,3,[Repeat Business Drivers],Repeat Business Drivers,Repeat Business Drivers,1,1,[1700.0],0.000048,0.000000,0.000041,0.000000,0.00,0.000000,0.000000,0.000000
1,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,42900.0,1950.000000,22,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,1,[1950.0],0.000352,0.047619,0.000287,0.000144,0.00,0.000000,0.000000,0.000000
2,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,12100.0,2016.666667,6,[High Value & Slow Moving],High Value & Slow Moving,High Value & Slow Moving,1,2,"[2350.0, 1950.0]",0.000096,0.000000,0.000082,0.000000,0.00,0.000000,0.000000,0.000000
3,Aeropress Coffee Maker,5321695592586,68700.0,3816.666667,18,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,2,"[4100.0, 3800.0]",0.000288,0.000000,0.000096,0.000000,0.00,0.000000,0.000000,0.000000
4,Aeropress Filter Paper,7736482594954,1040.0,520.000000,2,[Low Value & Slow Moving],Low Value & Slow Moving,Low Value & Slow Moving,1,1,[520.0],0.000032,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,259700.0,490.000000,510,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,[490.0],0.008157,0.051760,0.006609,0.003604,0.12,0.001784,0.666667,0.002899
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,150150.0,650.000000,195,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,[650.0],0.003119,0.085714,0.002394,0.002163,0.40,0.003567,0.166667,0.001449
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,23400.0,1300.000000,15,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,[1300.0],0.000240,0.000000,0.000205,0.000000,0.00,0.000000,0.000000,0.000000
911,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7542394617994,61850.0,1508.536585,41,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,3,"[1250.0, 1150.0, 1550.0]",0.000656,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [140]:
grouping

,product_name,product_id,Discount_List,Order_Count_Without_Discount,Order_Count_With_Discount
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,"[0.0, 0.0, 0.0]",3,0
1,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 260.0, 265...",16,6
2,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,"[0.0, 550.0, 1158.0, 0.0, 0.0, 0.0]",4,2
3,Aeropress Coffee Maker,5321695592586,"[0.0, 0.0, 0.0, 0.0, 252.0, 110.0, 0.0, 1050.0...",11,7
4,Aeropress Filter Paper,7736482594954,"[0.0, 0.0]",2,0
...,...,...,...,...,...
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,"[0.0, 490.0, 0.0, 0.0, 0.0, 220.0, 0.0, 0.0, 0...",51,459
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,"[325.0, 0.0, 1560.0, 650.0, 390.0, 0.0, 390.0,...",7,188
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,"[780.0, 1530.0, 780.0, 1530.0, 1530.0, 1950.0,...",0,15
911,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7542394617994,"[0.0, 0.0, 1250.0, 0.0, 0.0, 0.0, 1500.0, 475....",31,10


In [141]:
result

,product_id,product_name,Average_Price_With_Discount,Average_Price_Without_Discount
0,5320059781258,Thogarihunkal Estate Coffee - Aeropress / 250 gm,1236.600000,1731.50
1,5320059781258,Thogarihunkal Estate Coffee - Chemex / 250 gm,1135.000000,NaN
2,5320059781258,Thogarihunkal Estate Coffee - Cold Brew / 250 gm,1796.000000,1637.50
3,5320059781258,Thogarihunkal Estate Coffee - Cold Brew / 500 gm,2236.000000,NaN
4,5320059781258,Thogarihunkal Estate Coffee - Decoction Coffee...,1698.888889,1529.75
...,...,...,...,...
908,7792635084938,Brazil & Ethiopia Speciality Instant Coffee (P...,1446.500000,448.00
909,7792637018250,Colombia & Ethiopia Speciality Instant Coffee ...,NaN,748.00
910,7792637018250,Colombia & Ethiopia Speciality Instant Coffee ...,NaN,648.00
911,7792640884874,Brazil & Colombia Speciality Instant Coffee (P...,NaN,872.50


In [142]:
merged_result_1 = pd.merge(product_profile_final, grouping, on=['product_id', 'product_name'], how='left')

In [143]:
merged_result_2 = pd.merge(merged_result_1, result, on=['product_id', 'product_name'], how='left')

In [144]:
merged_result_2

,product_name,product_id,Total_Value,ASP,Total_Orders,Journey,First_Tag,Last_Tag,Tag_Count,Asp_Count,...,Cart_Penetration_for_2_transaction,Conversion_Rate_2_to_3,Cart_Penetration_for_3_transaction,Conversion_Rate_3_to_4,Cart_Penetration_for_4_transaction,Discount_List,Order_Count_Without_Discount,Order_Count_With_Discount,Average_Price_With_Discount,Average_Price_Without_Discount
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,5100.0,1700.000000,3,[Repeat Business Drivers],Repeat Business Drivers,Repeat Business Drivers,1,1,...,0.000000,0.00,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0]",3,0,NaN,1700.000000
1,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,42900.0,1950.000000,22,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,1,...,0.000144,0.00,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 260.0, 265...",16,6,9361.666667,2221.125000
2,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,12100.0,2016.666667,6,[High Value & Slow Moving],High Value & Slow Moving,High Value & Slow Moving,1,2,...,0.000000,0.00,0.000000,0.000000,0.000000,"[0.0, 550.0, 1158.0, 0.0, 0.0, 0.0]",4,2,3515.000000,2050.000000
3,Aeropress Coffee Maker,5321695592586,68700.0,3816.666667,18,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,2,...,0.000000,0.00,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 252.0, 110.0, 0.0, 1050.0...",11,7,6779.714286,4049.545455
4,Aeropress Filter Paper,7736482594954,1040.0,520.000000,2,[Low Value & Slow Moving],Low Value & Slow Moving,Low Value & Slow Moving,1,1,...,0.000000,0.00,0.000000,0.000000,0.000000,"[0.0, 0.0]",2,0,NaN,1395.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,259700.0,490.000000,510,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,0.003604,0.12,0.001784,0.666667,0.002899,"[0.0, 490.0, 0.0, 0.0, 0.0, 220.0, 0.0, 0.0, 0...",51,459,958.932462,590.392157
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,150150.0,650.000000,195,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,0.002163,0.40,0.003567,0.166667,0.001449,"[325.0, 0.0, 1560.0, 650.0, 390.0, 0.0, 390.0,...",7,188,1519.781915,1542.857143
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,23400.0,1300.000000,15,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,0.000000,0.00,0.000000,0.000000,0.000000,"[780.0, 1530.0, 780.0, 1530.0, 1530.0, 1950.0,...",0,15,3116.666667,NaN
911,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7542394617994,61850.0,1508.536585,41,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,3,...,0.000000,0.00,0.000000,0.000000,0.000000,"[0.0, 0.0, 1250.0, 0.0, 0.0, 0.0, 1500.0, 475....",31,10,2120.000000,2088.354839


In [145]:
merged_result_2.columns.to_list()

['product_name',
 'product_id',
 'Total_Value',
 'ASP',
 'Total_Orders',
 'Journey',
 'First_Tag',
 'Last_Tag',
 'Tag_Count',
 'Asp_Count',
 'Asp_Lst',
 'Cart_Penetration_Overall',
 'Conversion_Rate_1_to_2',
 'Cart_Penetration_for_1_transaction',
 'Cart_Penetration_for_2_transaction',
 'Conversion_Rate_2_to_3',
 'Cart_Penetration_for_3_transaction',
 'Conversion_Rate_3_to_4',
 'Cart_Penetration_for_4_transaction',
 'Discount_List',
 'Order_Count_Without_Discount',
 'Order_Count_With_Discount',
 'Average_Price_With_Discount',
 'Average_Price_Without_Discount']

In [146]:
final_df = merged_result_2[['product_name',
 'product_id',
 'Total_Value',
 'ASP',
 'Total_Orders',
 'Journey',
 'First_Tag',
 'Last_Tag',
 'Tag_Count',
 'Asp_Count',
 'Asp_Lst',
 'Cart_Penetration_Overall',
 'Conversion_Rate_1_to_2',
 'Cart_Penetration_for_1_transaction',
 'Cart_Penetration_for_2_transaction',
 'Conversion_Rate_2_to_3',
 'Cart_Penetration_for_3_transaction',
 'Conversion_Rate_3_to_4',
 'Cart_Penetration_for_4_transaction',
 'Discount_List',
 'Order_Count_With_Discount',
 'Order_Count_Without_Discount',
 'Average_Price_With_Discount',
 'Average_Price_Without_Discount']]

In [148]:
query_add = """select o.id as order_id,
            oli.product_id as product_id,
            oli.name as product_name,
            o.customer_id as user_id,
            a.address1 as customer_address,
                        CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
         ELSE 'Delivered'
     END as order_status,
            osa.address1 as delivery_address
            from orders o
            left join addresses a on a.customer_id = o.customer_id
            left join order_shipping_address osa on osa.order_id=o.id
            inner join order_line_item oli on oli.order_id = o.id"""

In [149]:
data1 = pd.read_sql(query_add, conn)

In [150]:
data1["order_status"].unique()

array(['Delivered', 'Cancelled'], dtype=object)

In [151]:
data1 = data1[data1['order_status']=='Delivered']

In [152]:
data1 = data1.dropna(subset=['customer_address'])

In [153]:
filtered_df = data1[data1["customer_address"] != "address not entered"]

In [154]:
filtered_df = filtered_df.drop_duplicates()
filtered_df

,order_id,product_id,product_name,user_id,customer_address,order_status,delivery_address
0,5341764845706,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.377996e+12,BUNGALOW NO. 71,Delivered,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE..."
2,5341750689930,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.704616e+12,"3/4 - 80 - EWS,",Delivered,"3/4 - 80 - EWS,"
4,5341737910410,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.801344e+12,Address:3/336-1 404-1 venugopala Swamy Street ...,Delivered,Address:3/336-1 404-1 venugopala Swamy Street ...
8,5341683515530,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.909104e+12,Esserpara Jumma Masjid,Delivered,Esserpara Jumma Masjid
13,5341679059082,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.837490e+12,BC 04\nchinad Kushalpora,Delivered,BC 04\nchinad Kushalpora
...,...,...,...,...,...,...,...
169597,5362703335562,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.739637e+12,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",Delivered,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1..."
169599,5362445484170,5.321648e+12,Single Malt Whiskey Barrel Aged Coffee - Whole...,6.968182e+12,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",Delivered,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves..."
169600,5361322983562,5.320286e+12,Baba Budangiri Estate Coffee - Whole Beans / 2...,6.955867e+12,"shubasree complex, 4A/18, dharmatala road, fla...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla..."
169601,5361322983562,5.320286e+12,Baba Budangiri Estate Coffee - Whole Beans / 2...,6.955867e+12,"188, Palash Society, Green Dr. Road, Shankar K...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla..."


In [155]:
customer_address = filtered_df.groupby(["user_id"]).agg(
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

In [156]:
customer_address["Delivery_Address_Counts"].iloc[20]

array([['B-1302, Bestech Park View Spa, Sector 47', 1]], dtype=object)

In [157]:
customer_address["user_id"].iloc[0]

2912876953738.0

In [158]:
import pandas as pd

# Assuming you have a DataFrame named customer_address
# with columns 'user_id' and 'Delivery_Address_Counts'

# Create a new DataFrame to store the results
result_rows = []

for i in range(len(customer_address)):
    user_id = customer_address.at[i, 'user_id']
    data = customer_address.at[i, 'Delivery_Address_Counts']

    # Find the index of the maximum count
    max_index = max(range(len(data)), key=lambda x: data[x][1])

    # Add the primary address to the result DataFrame
    result_rows.append({'user_id': user_id,
                        'address_type': 'primary',
                        'delivery_address': data[max_index][0],
                        'count': data[max_index][1]})

    # Add the rest of the addresses as gifting addresses to the result DataFrame
    result_rows.extend({'user_id': user_id,
                        'address_type': 'gifting',
                        'delivery_address': element,
                        'count': count} for j, (element, count) in enumerate(data) if j != max_index)

# Create the result DataFrame from the list of rows
result_df = pd.DataFrame(result_rows, columns=['user_id', 'address_type', 'delivery_address', 'count'])


In [159]:
result_df

,user_id,address_type,delivery_address,count
0,2.912877e+12,primary,LODHA Aurum Grande. Kanjurmarg East,6
1,2.912877e+12,gifting,Electronics City Phase 1 Electronic City,3
2,2.912877e+12,gifting,"269/127 Birhana, Devi dayal Marg , Lucknow",3
3,3.554806e+12,primary,"B109 Subhodaya Laurels,",60
4,3.554806e+12,gifting,#14 Kalpavriksha Apts Race Course Road,60
...,...,...,...,...
49580,6.970257e+12,primary,c-4/11 model town,1
49581,6.970262e+12,primary,do no 74_6/1_2,1
49582,6.970280e+12,primary,27/259 vasanthapeta,1
49583,6.970289e+12,primary,"141B, LIG, Suryanagar Phase 1, KHB Layout, Cha...",1


In [160]:
result_df_tag = result_df[['user_id', 'address_type', 'delivery_address']]
result_df_tag

,user_id,address_type,delivery_address
0,2.912877e+12,primary,LODHA Aurum Grande. Kanjurmarg East
1,2.912877e+12,gifting,Electronics City Phase 1 Electronic City
2,2.912877e+12,gifting,"269/127 Birhana, Devi dayal Marg , Lucknow"
3,3.554806e+12,primary,"B109 Subhodaya Laurels,"
4,3.554806e+12,gifting,#14 Kalpavriksha Apts Race Course Road
...,...,...,...
49580,6.970257e+12,primary,c-4/11 model town
49581,6.970262e+12,primary,do no 74_6/1_2
49582,6.970280e+12,primary,27/259 vasanthapeta
49583,6.970289e+12,primary,"141B, LIG, Suryanagar Phase 1, KHB Layout, Cha..."


In [161]:
final_df_with_tagging = pd.merge(filtered_df, result_df_tag, on=["user_id", "delivery_address"], how="left")

In [162]:
final_df_with_tagging

,order_id,product_id,product_name,user_id,customer_address,order_status,delivery_address,address_type
0,5341764845706,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.377996e+12,BUNGALOW NO. 71,Delivered,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE...",gifting
1,5341750689930,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.704616e+12,"3/4 - 80 - EWS,",Delivered,"3/4 - 80 - EWS,",primary
2,5341737910410,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.801344e+12,Address:3/336-1 404-1 venugopala Swamy Street ...,Delivered,Address:3/336-1 404-1 venugopala Swamy Street ...,primary
3,5341683515530,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.909104e+12,Esserpara Jumma Masjid,Delivered,Esserpara Jumma Masjid,primary
4,5341679059082,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.837490e+12,BC 04\nchinad Kushalpora,Delivered,BC 04\nchinad Kushalpora,primary
...,...,...,...,...,...,...,...,...
127793,5362703335562,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,6.739637e+12,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",Delivered,"FlT # 202 Pinnacle pride\n apts, Umanagar st-1...",primary
127794,5362445484170,5.321648e+12,Single Malt Whiskey Barrel Aged Coffee - Whole...,6.968182e+12,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",Delivered,"No.82, 2nd Main, 3rd Stage, 3rd Block, Basaves...",primary
127795,5361322983562,5.320286e+12,Baba Budangiri Estate Coffee - Whole Beans / 2...,6.955867e+12,"shubasree complex, 4A/18, dharmatala road, fla...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla...",gifting
127796,5361322983562,5.320286e+12,Baba Budangiri Estate Coffee - Whole Beans / 2...,6.955867e+12,"188, Palash Society, Green Dr. Road, Shankar K...",Delivered,"shubasree complex, 4A/18, dharmatala road, fla...",gifting


In [163]:

# Count the number of tags with 'primary' for each user_id
primary_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'primary'].groupby(['product_name', 'product_id']).size().reset_index(name='primary_count')

# Count the number of tags with 'gifting' for each user_id
gifting_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'gifting'].groupby(['product_name', 'product_id']).size().reset_index(name='gifting_count')

# Merge the two DataFrames on user_id
result = pd.merge(primary_count, gifting_count, on=['product_name', 'product_id'], how='outer').fillna(0)



In [164]:
result

,product_name,product_id,primary_count,gifting_count
0,Aerolatte Steam Free Milk Frother To-Go with S...,6.603461e+12,2.0,3.0
1,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,26.0,0.0
2,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,8.0,0.0
3,Aeropress Coffee Maker,5.321696e+12,17.0,8.0
4,Aeropress Filter Paper,7.736483e+12,2.0,0.0
...,...,...,...,...
908,Thippanahalli Estate Coffee - Cold Brew / 750 gm,5.320258e+12,0.0,5.0
909,Thippanahalli Estate Coffee - Hario V60 / 200 gm,5.320258e+12,0.0,3.0
910,Thippanahalli Estate Coffee - Pour Over / 1000 gm,5.320258e+12,0.0,3.0
911,Thogarihunkal Estate Coffee - Syphon Coffee / ...,5.320060e+12,0.0,4.0


In [165]:
final_df_final_new = pd.merge(final_df, result, on=["product_name", "product_id"], how="left")

In [166]:
final_df_final_new

,product_name,product_id,Total_Value,ASP,Total_Orders,Journey,First_Tag,Last_Tag,Tag_Count,Asp_Count,...,Cart_Penetration_for_3_transaction,Conversion_Rate_3_to_4,Cart_Penetration_for_4_transaction,Discount_List,Order_Count_With_Discount,Order_Count_Without_Discount,Average_Price_With_Discount,Average_Price_Without_Discount,primary_count,gifting_count
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,5100.0,1700.000000,3,[Repeat Business Drivers],Repeat Business Drivers,Repeat Business Drivers,1,1,...,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0]",0,3,NaN,1700.000000,2.0,3.0
1,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,42900.0,1950.000000,22,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,1,...,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 260.0, 265...",6,16,9361.666667,2221.125000,26.0,0.0
2,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,12100.0,2016.666667,6,[High Value & Slow Moving],High Value & Slow Moving,High Value & Slow Moving,1,2,...,0.000000,0.000000,0.000000,"[0.0, 550.0, 1158.0, 0.0, 0.0, 0.0]",2,4,3515.000000,2050.000000,8.0,0.0
3,Aeropress Coffee Maker,5321695592586,68700.0,3816.666667,18,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,2,...,0.000000,0.000000,0.000000,"[0.0, 0.0, 0.0, 0.0, 252.0, 110.0, 0.0, 1050.0...",7,11,6779.714286,4049.545455,17.0,8.0
4,Aeropress Filter Paper,7736482594954,1040.0,520.000000,2,[Low Value & Slow Moving],Low Value & Slow Moving,Low Value & Slow Moving,1,1,...,0.000000,0.000000,0.000000,"[0.0, 0.0]",0,2,NaN,1395.000000,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,259700.0,490.000000,510,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,0.001784,0.666667,0.002899,"[0.0, 490.0, 0.0, 0.0, 0.0, 220.0, 0.0, 0.0, 0...",459,51,958.932462,590.392157,562.0,93.0
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,150150.0,650.000000,195,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,0.003567,0.166667,0.001449,"[325.0, 0.0, 1560.0, 650.0, 390.0, 0.0, 390.0,...",188,7,1519.781915,1542.857143,268.0,70.0
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,23400.0,1300.000000,15,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,0.000000,0.000000,0.000000,"[780.0, 1530.0, 780.0, 1530.0, 1530.0, 1950.0,...",15,0,3116.666667,NaN,28.0,0.0
911,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7542394617994,61850.0,1508.536585,41,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,3,...,0.000000,0.000000,0.000000,"[0.0, 0.0, 1250.0, 0.0, 0.0, 0.0, 1500.0, 475....",10,31,2120.000000,2088.354839,46.0,4.0


In [167]:
final_df_final_new.to_csv("fdfnew.csv", index=False)

In [168]:
final_df_median_value_ASP = final_df['ASP'].median()
final_df['ASP_Category'] = pd.cut(final_df['ASP'],
                                                 bins=[float('-inf'), final_df_median_value_ASP, float('inf')],
                                                 labels=['Lower ASP', 'Higher ASP'])

In [169]:
final_df_median_value_cpfol = final_df['Cart_Penetration_for_1_transaction'].median()
final_df['Cart_Penetration_for_1_transaction_Category'] = pd.cut(final_df['Cart_Penetration_for_1_transaction'],
                                                 bins=[float('-inf'), final_df_median_value_cpfol, float('inf')],
                                                 labels=['Lower CP_First_Order_Level', 'Higher CP_First_Order_Level'])

In [170]:
final_df_median_value_cpsol = final_df['Cart_Penetration_for_2_transaction'].median()
final_df['Cart_Penetration_for_2_transaction_Category'] = pd.cut(final_df['Cart_Penetration_for_2_transaction'],
                                                 bins=[float('-inf'), final_df_median_value_cpsol, float('inf')],
                                                 labels=['Lower CP_Second_Order_Level', 'Higher CP_Second_Order_Level'])

In [182]:
final_df_median_value_cptol = final_df['Cart_Penetration_for_3_transaction'].median()
final_df['Cart_Penetration_for_3_transaction_Category'] = pd.cut(final_df['Cart_Penetration_for_3_transaction'],
                                                 bins=[float('-inf'), final_df_median_value_cptol, float('inf')],
                                                 labels=['Lower CP_Third_Order_Level', 'Higher CP_Third_Order_Level'])

In [183]:
final_df_median_value_cr12 = final_df['Conversion_Rate_1_to_2'].median()
final_df['Conversion_Rate_1_to_2_Category'] = pd.cut(final_df['Conversion_Rate_1_to_2'],
                                                 bins=[float('-inf'), final_df_median_value_cr12, float('inf')],
                                                 labels=['Lower CR_1_to_2', 'Higher CR_1_to_2'])

In [184]:
final_df_median_value_cr23 = final_df['Conversion_Rate_2_to_3'].median()
final_df['Conversion_Rate_2_to_3_Category'] = pd.cut(final_df['Conversion_Rate_2_to_3'],
                                                 bins=[float('-inf'), final_df_median_value_cr23, float('inf')],
                                                 labels=['Lower CR_2_to_3', 'Higher CR_2_to_3'])

In [185]:
final_df_median_value_cr34 = final_df['Conversion_Rate_3_to_4'].median()
final_df['Conversion_Rate_3_to_4_Category'] = pd.cut(final_df['Conversion_Rate_3_to_4'],
                                                 bins=[float('-inf'), final_df_median_value_cr34, float('inf')],
                                                 labels=['Lower CR_3_to_4', 'Higher CR_3_to_4'])

In [190]:
final_df_median_value_apwd = final_df['Average_Price_With_Discount'].median()
final_df['AP_Category'] = pd.cut(final_df['Average_Price_With_Discount'],
                                                 bins=[float('-inf'), final_df_median_value_apwd, float('inf')],
                                                 labels=['Lower Discount', 'Higher Discount'])

In [186]:
final_df.to_csv("final_df_new_df11.csv", index=False)

In [187]:
def component_1(final_df):
    df = pd.DataFrame()

    df['SKU_Name'] = final_df["product_name"]
    df['SKU_ID'] = final_df['product_id']
    df['Total_Orders'] = final_df['Total_Orders']
    df['ASP'] = final_df["ASP"]
    df["Tag"] = final_df["Last_Tag"]
    df["Cart_Penetration_at_First_Order_Level"] = final_df["Cart_Penetration_for_1_transaction"]
    df["Cart_Penetration_at_Second_Order_Level"] = final_df["Cart_Penetration_for_2_transaction"]
    df["Cart_Penetration_at_Third_Order_Level"] = final_df["Cart_Penetration_for_3_transaction"]
    df["Conversion_Rate_1_to_2"] = final_df["Conversion_Rate_1_to_2"]
    df["Conversion_Rate_2_to_3"] = final_df["Conversion_Rate_2_to_3"]
    df["Conversion_Rate_3_to_4"] = final_df["Conversion_Rate_3_to_4"]
    df["Percent_Orders_Without_Discount"] = final_df["Order_Count_Without_Discount"]/final_df["Total_Orders"]
    df["Average_Price_With_Discount"] = final_df["Average_Price_With_Discount"]

    return df

result = component_1(final_df)

In [191]:
final_df

,product_name,product_id,Total_Value,ASP,Total_Orders,Journey,First_Tag,Last_Tag,Tag_Count,Asp_Count,...,Average_Price_With_Discount,Average_Price_Without_Discount,ASP_Category,Cart_Penetration_for_1_transaction_Category,Cart_Penetration_for_2_transaction_Category,Cart_Penetration_for_3_transaction_Category,Conversion_Rate_1_to_2_Category,Conversion_Rate_2_to_3_Category,Conversion_Rate_3_to_4_Category,AP_Category
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,5100.0,1700.000000,3,[Repeat Business Drivers],Repeat Business Drivers,Repeat Business Drivers,1,1,...,NaN,1700.000000,Higher ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,NaN
1,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,42900.0,1950.000000,22,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,1,...,9361.666667,2221.125000,Higher ASP,Higher CP_First_Order_Level,Higher CP_Second_Order_Level,Lower CP_Third_Order_Level,Higher CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,Higher Discount
2,Aerolatte Steam Free Milk Frother With Stand -...,6603459690634,12100.0,2016.666667,6,[High Value & Slow Moving],High Value & Slow Moving,High Value & Slow Moving,1,2,...,3515.000000,2050.000000,Higher ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,Higher Discount
3,Aeropress Coffee Maker,5321695592586,68700.0,3816.666667,18,"[High Value & Slow Moving, New Business Drivers]",High Value & Slow Moving,New Business Drivers,2,2,...,6779.714286,4049.545455,Higher ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,Higher Discount
4,Aeropress Filter Paper,7736482594954,1040.0,520.000000,2,[Low Value & Slow Moving],Low Value & Slow Moving,Low Value & Slow Moving,1,1,...,NaN,1395.000000,Lower ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,259700.0,490.000000,510,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,958.932462,590.392157,Lower ASP,Higher CP_First_Order_Level,Higher CP_Second_Order_Level,Higher CP_Third_Order_Level,Higher CR_1_to_2,Higher CR_2_to_3,Higher CR_3_to_4,Lower Discount
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,150150.0,650.000000,195,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,1519.781915,1542.857143,Lower ASP,Higher CP_First_Order_Level,Higher CP_Second_Order_Level,Higher CP_Third_Order_Level,Higher CR_1_to_2,Higher CR_2_to_3,Higher CR_3_to_4,Lower Discount
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,23400.0,1300.000000,15,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,1,...,3116.666667,NaN,Higher ASP,Higher CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,Higher Discount
911,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7542394617994,61850.0,1508.536585,41,"[Low Value & Slow Moving, New Business Drivers]",Low Value & Slow Moving,New Business Drivers,2,3,...,2120.000000,2088.354839,Higher ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,Higher Discount


In [192]:
def component_2(final_df_new_upload, level):
    df = pd.DataFrame()
    first_part, second_part = level.split(" ")
    if second_part == "ASP":
      filtered_df = final_df[final_df["ASP_Category"] == level]
    if second_part == "CP_First_Order_Level":
      filtered_df = final_df[final_df["Cart_Penetration_for_1_transaction_Category"] == level]
    if second_part == "CP_Second_Order_Level":
      filtered_df = final_df[final_df["Cart_Penetration_for_2_transaction_Category"] == level]
    if second_part == "CP_Third_Order_Level":
      filtered_df = final_df[final_df["Cart_Penetration_for_3_transaction_Category"] == level]
    if second_part == "CR_1_to_2":
      filtered_df = final_df[final_df["Conversion_Rate_1_to_2_Category"] == level]
    if second_part == "CR_2_to_3":
      filtered_df = final_df[final_df["Conversion_Rate_2_to_3_Category"] == level]
    if second_part == "CR_3_to_4":
      filtered_df = final_df[final_df["Conversion_Rate_3_to_4_Category"] == level]
    if second_part == "Discount":
      filtered_df = final_df[final_df["AP_Category"] == level]


    df['SKU_Name'] = filtered_df["product_name"]
    df['SKU_ID'] = filtered_df['product_id']
    df['Total_Orders'] = filtered_df['Total_Orders']
    df['ASP'] = filtered_df["ASP"]
    df["Tag"] = filtered_df["Last_Tag"]
    df["Cart_Penetration_at_First_Order_Level"] = filtered_df["Cart_Penetration_for_1_transaction"]
    df["Cart_Penetration_at_Second_Order_Level"] = filtered_df["Cart_Penetration_for_2_transaction"]
    df["Cart_Penetration_at_Third_Order_Level"] = filtered_df["Cart_Penetration_for_3_transaction"]
    df["Conversion_Rate_1_to_2"] = filtered_df["Conversion_Rate_1_to_2"]
    df["Conversion_Rate_2_to_3"] = filtered_df["Conversion_Rate_2_to_3"]
    df["Conversion_Rate_3_to_4"] = filtered_df["Conversion_Rate_3_to_4"]
    df["Percent_Orders_Without_Discount"] = filtered_df["Order_Count_Without_Discount"]/filtered_df["Total_Orders"]
    df["Average_Price_With_Discount"] = filtered_df["Average_Price_With_Discount"]

    return df

result = component_2(final_df, "Lower ASP")

In [193]:
result

,SKU_Name,SKU_ID,Total_Orders,ASP,Tag,Cart_Penetration_at_First_Order_Level,Cart_Penetration_at_Second_Order_Level,Cart_Penetration_at_Third_Order_Level,Conversion_Rate_1_to_2,Conversion_Rate_2_to_3,Conversion_Rate_3_to_4,Percent_Orders_Without_Discount,Average_Price_With_Discount
4,Aeropress Filter Paper,7736482594954,2,520.000000,Low Value & Slow Moving,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN
6,Aeropress Rubber Seal,6575327805578,54,650.000000,New Business Drivers,0.000287,0.000000,0.000000,0.000000,0.000000,0.000000,0.870370,2351.857143
32,Annanas: PineApple Fermented Coffee - Aeropres...,5411502555274,61,619.114754,Hero,0.000739,0.000433,0.000595,0.055556,0.333333,1.000000,0.049180,1431.017241
33,Annanas: PineApple Fermented Coffee - Chemex /...,5411502555274,3,630.000000,Repeat Business Drivers,0.000041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1266.666667
34,Annanas: PineApple Fermented Coffee - Cold Bre...,5411502555274,56,603.017857,Hero,0.000616,0.001153,0.001189,0.177778,0.250000,0.500000,0.142857,2363.645833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,Vanilla Cold Brew - Easy Brew Cold Brew Bags (...,6991108374666,29,581.034483,Hero,0.000397,0.002739,0.000000,0.655172,0.000000,0.000000,0.068966,1268.518519
904,Vanilla Cold Brew - Easy Brew Cold Brew Bags -...,6991108374666,740,489.054054,New Business Drivers,0.009551,0.008218,0.005351,0.081662,0.157895,0.333333,0.062162,915.216138
905,Vanilla Cold Brew - Easy Brew Cold Brew Bags -...,6991108374666,117,644.871795,Hero,0.001314,0.002739,0.003567,0.197917,0.315789,0.666667,0.025641,1483.491228
908,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7570590236810,510,490.000000,New Business Drivers,0.006609,0.003604,0.001784,0.051760,0.120000,0.666667,0.100000,958.932462


In [189]:
final_df.head(1)

,product_name,product_id,Total_Value,ASP,Total_Orders,Journey,First_Tag,Last_Tag,Tag_Count,Asp_Count,...,Order_Count_Without_Discount,Average_Price_With_Discount,Average_Price_Without_Discount,ASP_Category,Cart_Penetration_for_1_transaction_Category,Cart_Penetration_for_2_transaction_Category,Cart_Penetration_for_3_transaction_Category,Conversion_Rate_1_to_2_Category,Conversion_Rate_2_to_3_Category,Conversion_Rate_3_to_4_Category
0,Aerolatte Steam Free Milk Frother To-Go with S...,6603461296266,5100.0,1700.0,3,[Repeat Business Drivers],Repeat Business Drivers,Repeat Business Drivers,1,1,...,3,NaN,1700.0,Higher ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4
